In [17]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import sys
print(sys.executable)
sys.path.insert(1, '../src/')
from config import raw_data_path, univariate_data_path, processed_data_path, models_path
from preprocessing_modules import create_time_windows_with_labels, create_time_windows_with_metadata
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
import matplotlib.pyplot as plt
from FCMAE_model import FCMAE
from fastai.callback.tracker import SaveModelCallback, EarlyStoppingCallback


/home/nwertheim/miniconda3/bin/python


In [18]:
# Define FCMAE Hyperparameters:
batch_size = 16
num_blocks = 5
kernel_size = 7
base_dim = 32
learning_rate = 0.00016938000495408888
input_dimension = 512

num_epochs = 10

In [19]:

import torch
import numpy as np
import os
from torch.utils.data import TensorDataset, DataLoader, Subset
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
import optuna
from fastai.learner import Learner
from fastai.data.core import DataLoaders
from fastai.metrics import accuracy
from tsai.models.ResNet import ResNet
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from fastai.optimizer import SGD
# Load training data
train_dir = os.path.join(univariate_data_path, "target_univariate_no_PCA_train_2.npy")
train_data = np.load(train_dir, allow_pickle=True)
train_windows, train_labels, train_meta = create_time_windows_with_metadata(train_data)

# Convert to tensors
train_windows_tensor = torch.tensor(train_windows, dtype=torch.float32).unsqueeze(1)  # shape: (N, 1, 12000)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_labels), y=train_labels)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)

# Prepare dataset
full_dataset = TensorDataset(train_windows_tensor, train_labels_tensor)

In [20]:



# Load the pre-trained encoder
path = os.path.join(models_path, 'FCMAE_encoder_no_PCA_gpu_normalized_correct.pth')

# Load the full FCMAE model
fcmae = FCMAE(in_channels=1, base_dim=base_dim, num_blocks=num_blocks, kernel_size=kernel_size)


# Load the saved weights into the full model
state_dict = torch.load(path)
fcmae.load_state_dict(state_dict)

# Extract only the encoder from the FCMAE model
encoder = fcmae.encoder
# Set the encoder to evaluation mode and freeze the encoder parameters
encoder.eval()
for param in encoder.parameters():
    param.requires_grad = False  # Freeze the encoder

print(encoder)

Sequential(
  (0): Conv1d(1, 32, kernel_size=(7,), stride=(2,), padding=(3,))
  (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Conv1d(32, 64, kernel_size=(7,), stride=(2,), padding=(3,))
  (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): Conv1d(64, 128, kernel_size=(7,), stride=(2,), padding=(3,))
  (7): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU()
  (9): Conv1d(128, 256, kernel_size=(7,), stride=(2,), padding=(3,))
  (10): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): ReLU()
  (12): Conv1d(256, 512, kernel_size=(7,), stride=(2,), padding=(3,))
  (13): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (14): ReLU()
)


In [21]:
'''tuneable optimizer, lower LR-range, early stopping'''
from fastai.learner import Metric
from sklearn.metrics import average_precision_score
from fastai.optimizer import SGD, Adam


# Custom Average Precision Metric
class AveragePrecision(Metric):
    def __init__(self):
        self.pred = []
        self.target = []

    def reset(self):
        self.pred, self.target = [], []

    def accumulate(self, learn):
        preds = learn.pred.detach().softmax(dim=-1)[:, 1]
        targs = learn.y.detach()
        self.pred.append(preds.cpu())
        self.target.append(targs.cpu())

    @property
    def value(self):
        preds = torch.cat(self.pred).numpy()
        targs = torch.cat(self.target).numpy()
        return average_precision_score(targs, preds)

    @property
    def name(self): return "avg_precision"

def objective(trial):
    lr = trial.suggest_loguniform('lr', 1e-7, 1e-6)
    optimizer_name = trial.suggest_categorical('optimizer', ['SGD', 'Adam'])

    # Map optimizer name to fastai optimizer function
    opt_func_map = {
    'SGD': SGD,
    'Adam': Adam,
    }

    opt_func = opt_func_map[optimizer_name]

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    avg_precisions = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(train_windows_tensor, train_labels_tensor)):
        train_subset = Subset(full_dataset, train_idx)
        val_subset = Subset(full_dataset, val_idx)

        train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=64, shuffle=False)
        dls = DataLoaders(train_loader, val_loader)

        class FCMAEClassifier(nn.Module):
            def __init__(self, encoder):
                super().__init__()
                self.encoder = encoder
                self.classifier = ResNet(input_dimension, 2)

            def forward(self, x):
                with torch.no_grad():
                    x = self.encoder(x)
                return self.classifier(x)

        model = FCMAEClassifier(encoder).to(device)
        loss_func = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)

        learn = learn = Learner(
            dls,
            model,
            loss_func=loss_func,
            opt_func=opt_func,
            metrics=accuracy,
            cbs=[
                EarlyStoppingCallback(monitor='valid_loss', patience=3),
            ]
        )

        learn.fit_one_cycle(num_epochs, lr)
        ap = learn.validate()[1]  # [loss, avg_precision]
        avg_precisions.append(ap)

    return np.mean(avg_precisions)  # maximize this

# Run Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=25)

print("Best trial:")
print(study.best_trial)


# Save the study
import joblib
joblib.dump(study, "FCMAE_ResNet_tune.pkl")


[I 2025-05-15 15:28:41,502] A new study created in memory with name: no-name-9275e347-adfc-4c74-9990-804dd59edcac


epoch,train_loss,valid_loss,accuracy,time
0,0.708208,0.712560,0.489899,00:01
1,0.707969,0.712253,0.494949,00:01
2,0.707969,0.708399,0.502525,00:01
3,0.710726,0.713041,0.457071,00:01
4,0.709264,0.714795,0.471212,00:01
5,0.710926,0.709249,0.504545,00:01


No improvement since epoch 2: early stopping


epoch,train_loss,valid_loss,accuracy,time
0,0.730961,0.733514,0.530808,00:01
1,0.731975,0.745904,0.522222,00:01
2,0.733048,0.742316,0.532323,00:01
3,0.733490,0.742832,0.525758,00:01


No improvement since epoch 0: early stopping


...

epoch,train_loss,valid_loss,accuracy,time
0,0.722618,0.670974,0.662121,00:01
1,0.724840,0.668389,0.662121,00:01
2,0.720659,0.675238,0.662121,00:01
3,0.725705,0.673073,0.662121,00:01
4,0.722050,0.669859,0.662121,00:01


No improvement since epoch 1: early stopping


[I 2025-05-15 15:48:45,800] Trial 24 finished with value: 0.5165656685829163 and parameters: {'lr': 5.888571506170028e-07, 'optimizer': 'SGD'}. Best is trial 22 with value: 0.6014141321182251.


Best trial:
FrozenTrial(number=22, state=1, values=[0.6014141321182251], datetime_start=datetime.datetime(2025, 5, 15, 15, 46, 51, 81681), datetime_complete=datetime.datetime(2025, 5, 15, 15, 47, 26, 131767), params={'lr': 3.1262471855603347e-07, 'optimizer': 'SGD'}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'lr': FloatDistribution(high=1e-06, log=True, low=1e-07, step=None), 'optimizer': CategoricalDistribution(choices=('SGD', 'Adam'))}, trial_id=22, value=None)


['FCMAE_ResNet_tune.pkl']

In [22]:
print("Best trial:")
print(study.best_trial)


Best trial:
FrozenTrial(number=22, state=1, values=[0.6014141321182251], datetime_start=datetime.datetime(2025, 5, 15, 15, 46, 51, 81681), datetime_complete=datetime.datetime(2025, 5, 15, 15, 47, 26, 131767), params={'lr': 3.1262471855603347e-07, 'optimizer': 'SGD'}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'lr': FloatDistribution(high=1e-06, log=True, low=1e-07, step=None), 'optimizer': CategoricalDistribution(choices=('SGD', 'Adam'))}, trial_id=22, value=None)


In [23]:
# '''MY DATA'''
# train_dir = os.path.join(univariate_data_path, "target_univariate_no_PCA_train2.npy")
# train_data = np.load(train_dir, allow_pickle=True)
# # train_data = pd.DataFrame(train_data)
# test_dir = os.path.join(univariate_data_path, "target_univariate_no_PCA_test2.npy")
# test_data = np.load(test_dir, allow_pickle=True)
# # test_data = pd.DataFrame(test_data)
# print(len(train_data))
# print(len(test_data))
# print(train_data[0])
# from torch.utils.data import DataLoader, TensorDataset, random_split
# from sklearn.utils.class_weight import compute_class_weight

# from collections import Counter

# # Example dataset with windows and labels
# train_windows, train_labels = create_time_windows_with_labels(train_data)
# test_windows, test_labels = create_time_windows_with_labels(test_data)

# # Count label distribution
# train_label_counts = Counter(train_labels)
# test_label_counts = Counter(test_labels)

# # Print the counts
# print("Train label distribution:")
# print(f"  Term (0): {train_label_counts[0]}")
# print(f"  Preterm (1): {train_label_counts[1]}")

# print("\nTest label distribution:")
# print(f"  Term (0): {test_label_counts[0]}")
# print(f"  Preterm (1): {test_label_counts[1]}")

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# # Convert to tensors
# train_windows_tensor = torch.tensor(train_windows, dtype=torch.float32)
# train_labels_tensor = torch.tensor(train_labels, dtype=torch.float32)
# test_windows_tensor = torch.tensor(test_windows, dtype=torch.float32)
# test_labels_tensor = torch.tensor(test_labels, dtype=torch.float32)

# train_windows_tensor = train_windows_tensor.unsqueeze(1)  # (1071, 1, 12000)
# test_windows_tensor = test_windows_tensor.unsqueeze(1)    # (899, 1, 12000)

# train_labels_tensor = train_labels_tensor.long()
# test_labels_tensor = test_labels_tensor.long()


# print(train_labels_tensor.shape)

# train_dataset = TensorDataset(train_windows_tensor, train_labels_tensor)
# test_dataset = TensorDataset(test_windows_tensor, test_labels_tensor)

# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



In [24]:
# '''CLASS WEIGHTS'''
# from tsai.models.ResNet import ResNet
# from tsai.models import ResNet
# from tsai.models.ResNet import ResNet
# from fastai.metrics import accuracy
# from fastai.data.core import DataLoaders
# from fastai.learner import Learner
# import torch

# class FCMAEClassifier(nn.Module):
#     def __init__(self, encoder):
#         super().__init__()
#         self.encoder = encoder
#         self.classifier = ResNet(2048, 2)  # 2048 channels from encoder, 2 output classes

#     def forward(self, x):
#         with torch.no_grad():  # freeze encoder
#             x = self.encoder(x)
#         return self.classifier(x)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Wrap your DataLoaders and set device
# dls = DataLoaders.from_dsets(
#     train_dataset,
#     test_dataset,
#     bs=32,
#     shuffle=True,
#     num_workers=0 
# )

# model = FCMAEClassifier(encoder)
# model.to(device)

# # Calculate class weights
# class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_labels), y=train_labels)
# class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)

# # Use weighted loss
# loss_func = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)

# # Replace your loss function in the learner
# learn = Learner(dls, model, loss_func=loss_func, metrics=accuracy)
# learn.fit_one_cycle(20, 1e-6)

# learn.recorder.plot_loss()  # Plot loss curves (training and validation loss)
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.metrics import confusion_matrix
# import torch

# # Get predictions and true labels
# preds, targs = learn.get_preds(dl=learn.dls.valid)

# # Convert predictions to class labels (argmax for multi-class classification)
# pred_labels = preds.argmax(dim=1)

# # Compute confusion matrix
# cm = confusion_matrix(targs, pred_labels)

# # Plot confusion matrix
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False, 
#             xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
# plt.title('Confusion Matrix')
# plt.xlabel('Predicted Label')
# plt.ylabel('True Label')
# plt.show()

# from sklearn.metrics import (
#     accuracy_score,
#     precision_score,
#     recall_score,
#     f1_score,
#     average_precision_score,
#     roc_auc_score
# )

# # Convert to numpy arrays if needed
# true_labels = targs.cpu().numpy()
# pred_labels = pred_labels.cpu().numpy()
# pred_probs = preds[:, 1].cpu().numpy()  # Probability of class 1

# # Compute metrics
# accuracy = accuracy_score(true_labels, pred_labels)
# precision = precision_score(true_labels, pred_labels, zero_division=0)
# recall = recall_score(true_labels, pred_labels, zero_division=0)
# f1 = f1_score(true_labels, pred_labels, zero_division=0)
# ap = average_precision_score(true_labels, pred_probs)
# auc = roc_auc_score(true_labels, pred_probs)

# # Print results
# print(f"Accuracy: {accuracy:.4f}")
# print(f"Precision: {precision:.4f}")
# print(f"Recall: {recall:.4f}")
# print(f"F1 Score: {f1:.4f}")
# print(f"Average Precision (AP): {ap:.4f}")
# print(f"Area Under ROC Curve (AUC): {auc:.4f}")

